In [ ]:
#!pip install xmlschema
#!pip install lxml


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [14]:
import xmlschema
from xmlschema import limits
import gzip
import os

In [15]:

limits.MAX_XML_ELEMENTS = 10 ** 10

In [ ]:
workspace_path = os.path.join('scratch', 'NPD Data Dictionaries & Initial File Share (Nov 2025)')
xsd_path = os.path.join(workspace_path, 'PECOS UGE OnBoarding Documents', 'PECOS UGE v6.xsd')
xml_path = os.path.join(workspace_path, 'P.UGE.PECOS.INITIAL.D251115.T0015300.gz')

In [9]:
schema = xmlschema.XMLSchema(xsd_path)

In [16]:
with gzip.open(xml_path, 'rb') as f:
    schema_dict = schema.decode(f, lazy = True)

: 